# Clases y objetos

Acá una explicación tomada de [Brilliant](https://brilliant.org/wiki/classes-oop/):

> In object-oriented programming, a class is a blueprint for creating objects (a particular data structure), providing initial values for state (member variables or attributes), and implementations of behavior (member functions or methods).

Acá una de [Wikipedia](https://simple.wikipedia.org/wiki/Class_(programming)):

> A class is written by a programmer in a defined structure to create an object (computer science) in an object oriented programming language. It defines a set of properties and methods that are common to all objects of one type.

Acá otra de [IIT Kanpur](http://www.iitk.ac.in/esc101/05Aug/tutorial/java/concepts/class.html):

> A class is a blueprint that defines the variables and the methods common to all objects of a certain kind

Ahora un ejemplo tomado de [Dummies.com](https://www.dummies.com/programming/java/understanding-classes-and-objects/)

> A class is like a blueprint for a kind of house in a housing development. An object is like a particular house. The blueprint says things like “Each house’s living room has its own color paint.” A particular house has red paint, or white paint, or some other color paint.

Honestamente, si nunca has tenido contacto con programación orientada a objetos, te recomiendo visitar todas las fuentes que cité, con media hora de lectura. mas los ejemplos y talleres que haremos en clase, sabrás todo lo necesario.

Finalmente, acá te dejo un par de lecturas para entender un poco más a profundidad la programación orientada a objetos, todos los conceptos los usaremos en el curso:

- [What is Object Oriented Programming? OOP Explained in Depth](https://www.educative.io/blog/object-oriented-programming)
- [How to explain object-oriented programming concepts to a 6-year-old](https://www.freecodecamp.org/news/object-oriented-programming-concepts-21bb035f7260/)

En esta ocasión vamos a partir de un caso práctico para usar programación orientada objetos. Usaremos el ejercicio de probabilidad de 2.2.14 de [Richard J Larsen and Morris L Marx: An Introduction to Mathematical Statistics and Its Applications](https://www.amazon.com/Introduction-Mathematical-Statistics-Its-Applications/dp/0321693949). El ejercicio dice lo siguiente:

> A probability-minded despot offers a convicted murderer a final chance to gain his release. The prisoner is given twenty chips, ten white and ten black. All twenty are to be placed into two urns, according to any allocation scheme the prisoner wishes, with the one proviso being that each urn contain at least one chip. The executioner will then pick one of the two urns at random and from that urn, one chip at random. If the chip selected is white, the prisoner will be set free; if it is black, he “buys the farm.” Characterize the sample space describing the prisoner’s possible allocation options. (Intuitively, which allocation affords the prisoner the greatest chance of survival?)

Es importante resaltar que el libro mencionado no tiene nada que ver con programación, pero me pareció un interesante ejercicio para mostrar cómo podemos sacar provecho de la programación orientada objetos.

In [1]:
import random
import itertools
from collections import Counter

## Modelar el problema utilizando OOP

El primer elemento del que nos hablan son los `chips`, de los que sabemos únicamente su color; esta es la única propiedad importante, por lo que nos limitaremos a representar chips con un tipo `str`. 

Empecemos modelando el elemento más importante de todo el experimento: las urnas (`urn`). De cada urna sabemos lo siguiente:

- Tiene un conjunto de `chips`.
- Tiene un número definido de `chips` blancos.
- Tiene un número definido de `chips` negros.
- Hay una probabilidad de sacar blanco o negro de acuerdo al número de `chips` de cada color.

In [2]:
class Urn:
    """A container of black and white chips.
    
    Args:
        chips (list[str]): chips to put in the urn, must
            be `"white"` or `"black"` only.
        
    Attributes:
        num_chips (int): total number of chips
        num_white_chips (int): number of white chips in urn.
        num_black_chips (int): number of black chips in urn.
        prob_white (float): probability of getting a white chip
            out of the urn.
        prob_black (float): probability of getting a black chip
            out of the urn.
            
    """
    _VALID_CHIPS = frozenset(["white", "black"])
    def __init__(self, chips):
        if set(chips).union(self._VALID_CHIPS) != self._VALID_CHIPS:
            msg = "this urn only support {}, got {}"
            raise ValueError(msg.format(
                self._VALID_CHIPS, set(chips)
            ))
            
        self._chips = chips
        random.shuffle(self._chips)
        
    def sample(self):
        """Returns a random chip from the urn. """
        return self._chips.pop()
    
    @property
    def num_chips(self):
        return len(self._chips)
    
    @property
    def num_white_chips(self):
        return Counter(self._chips)["white"]
    
    @property
    def num_black_chips(self):
        return self.num_chips - self.num_white_chips

    @property
    def prob_white(self):
        """float: Returns the probability of taking a white chip. """
        return self.num_white_chips / self.num_chips

    @property
    def prob_black(self):
        """float: Returns the probability of taking a black chip. """
        return self.num_black_chips / self.num_chips
    
    def __repr__(self):
        return f"Urn({self.num_white_chips}W.{self.num_black_chips}B)"

Hasta ahora no hemos ejecutado nada de código, lo único que hemos hecho es definir cuáles son los elementos que describen una urna (propiedades) y lo que se puede hacer con una de ellas (métodos). A continuación, crearemos una instancia (un objeto) de la clase `Urn` y extraeremos sus propiedades.

In [3]:
u = Urn(["white", "black", "black"])
print(f"Number of chips in urn: {u.num_chips}")
print(f"Number of white chips in urn: {u.num_white_chips}")
print(f"Number of black chips in urn: {u.num_black_chips}")
print(f"Probability of randomly taking a white chip out: {u.prob_white}")
print(f"Probability of randomly taking a black chip out: {u.prob_black}")
assert round(u.prob_white + u.prob_black ) == 1 # probabilities shuold sum up to one

Number of chips in urn: 3
Number of white chips in urn: 1
Number of black chips in urn: 2
Probability of randomly taking a white chip out: 0.3333333333333333
Probability of randomly taking a black chip out: 0.6666666666666666


Miremos qué pasaría si intetáramos crear una urna con colores no válidos

In [4]:
Urn(["pink", "blue", "white", "black"])

ValueError: this urn only support frozenset({'white', 'black'}), got {'white', 'black', 'blue', 'pink'}

gracias a esto, podemos restringir bastante nuestro problema, para evitar incluir errores sin darnos cuenta. Piensa en qué pasaría si tratamos de utilizar un `chip` azul: los cálculos de probabilidad serían inválidos. 

¿Cómo sacar un element aleatoriamente de nuestra urna?

Usar el método `sample`!

In [5]:
u.sample()

'white'

Finalmente, la implementación del método `__repr__` nos permite imprimir nuestros objetos de una forma útil (trata eliminar ese método y mira el resultado de la siguiente línea):

In [6]:
u

Urn(0W.2B)

Nota que hay un `chip` blanco menos en la urna comparando con el inicio.

Ahora vamos a la parte más interesante del problema, vamos a modelar cómo luce una asignación de `chips` blancos y negros en las dos urnas planteadas. Recordemos que de una asignación, solo nos importa la probabilidad de salir libre.

In [7]:
class Allocation:
    """A distribution of allowed chips in two urns.
    
    Note:
        The we enforce that all chips for the problem are
        placed in one of the two urns.
    
    Args:
        urn1 (Urn): An urn, it must have at least 1 chip.
        urn2 (Urn): An urn, it must have at least 1 chip.
        
    Attributes:
        prob_release (float): probability of release from a
            given allocation of chips.
            
    """
    _TOTAL_WHITE_CHIPS = 10
    _TOTAL_BLACK_CHIPS = 10
    _ALL_CHIPS = (
        ["white"] * _TOTAL_WHITE_CHIPS
        + ["black"] * _TOTAL_BLACK_CHIPS
    )
    def __init__(self, urn1, urn2):
        if not self._is_valid(urn1, urn2):
            raise ValueError(f"invalid allocation {urn1}, {urn2}")
        self._urn1 = urn1
        self._urn2 = urn2

    @classmethod
    def make_random_allocation(cls):
        """ This method allows you to create new random allocations. """
        total_in_urn1 = random.randrange(1, 20)
        urn1 = Urn(random.sample(cls._ALL_CHIPS, total_in_urn1))
        urn2 = Urn(
            ["white"] * (cls._TOTAL_WHITE_CHIPS - urn1.num_white_chips)
            + ["black"] * (cls._TOTAL_BLACK_CHIPS - urn1.num_black_chips)
        )
        return cls(urn1, urn2)

    @property
    def prob_release(self):
        return 0.5 * (self._urn1.prob_white + self._urn2.prob_white)
    
    def _is_valid(self, urn1, urn2):
        cond1 = urn1.num_chips >= 1 and urn2.num_chips >= 1
        cond2 = urn1.num_white_chips + urn2.num_white_chips == self._TOTAL_WHITE_CHIPS
        cond3 = urn1.num_black_chips + urn2.num_black_chips == self._TOTAL_BLACK_CHIPS
        return cond1 and cond2 and cond3

    def __repr__(self):
        return f"Allocation({self._urn1}, {self._urn2})"

Acá utilizamos algo diferente: `@classmethod` . A diferencia de los demás métodos, este corresponde a uno de la clase y no de los objetos, suelen ser usados para crear objetos de la clase en que se definen.

In [8]:
allocation = Allocation.make_random_allocation()
allocation

Allocation(Urn(6W.5B), Urn(4W.5B))

In [9]:
allocation.prob_release

0.4949494949494949

Con esta forma de modelar el problema, nos resulta bastante claro hacer una simulación, para extraer respuestas importantes para el problema planteado.

**¿Cuál es la probabilidad de salir libre con asignación aleatoria?**

In [10]:
def freedom_probability_of_random_allocation(num_trials):
    return sum(
        Allocation.make_random_allocation().prob_release
        for _ in range(num_trials)
    )/num_trials

In [11]:
freedom_probability_of_random_allocation(10000)

0.49890640735747593

**¿Qué asignación maximiza la probabilidad de salir libre?**

In [12]:
def best_allocation():
    best_allocation = Allocation.make_random_allocation()
    for i, j in itertools.product(range(0, 10), repeat=2):
        if 0 < i + j < 19:
            num_white_in_urn1 = i
            num_black_in_urn1 = j
            num_white_in_urn2 = 10 - num_white_in_urn1
            num_black_in_urn2 = 10 - num_black_in_urn1

            urn1 = Urn(
                ["white"] * num_white_in_urn1
                + ["black"] * num_black_in_urn1
            )
            urn2 = Urn(
                ["white"] * num_white_in_urn2
                + ["black"] * num_black_in_urn2
            )
            allocation = Allocation(urn1, urn2)
            if allocation.prob_release > best_allocation.prob_release:
                best_allocation = allocation
    return best_allocation

In [13]:
allocation = best_allocation()
msg = "Best allocation is {}, with release probability of {}"
print(msg.format(allocation, allocation.prob_release))

Best allocation is Allocation(Urn(1W.0B), Urn(9W.10B)), with release probability of 0.7368421052631579


In [14]:
def approximate_best_allocation(num_trials):
    best_allocation = Allocation.make_random_allocation()
    for _ in range(num_trials):
        num_white_in_urn1 = random.randint(0, 10)
        num_black_in_urn1 = 10 - num_white_in_urn1
        num_white_in_urn2 = 10 - num_white_in_urn1
        num_black_in_urn2 = 10 - num_black_in_urn1
        urn1 = Urn(
            ["white"] * num_white_in_urn1
            + ["black"] * num_black_in_urn1
        )
        urn2 = Urn(
            ["white"] * num_white_in_urn2
            + ["black"] * num_black_in_urn2
        )
        allocation = Allocation(urn1, urn2)
        if allocation.prob_release > best_allocation.prob_release:
            best_allocation = allocation
    return best_allocation

In [15]:
allocation = approximate_best_allocation(5000)
msg = "Approximate best allocation is {}, with release probability of {}"
print(msg.format(allocation, allocation.prob_release))

Approximate best allocation is Allocation(Urn(4W.6B), Urn(6W.4B)), with release probability of 0.5
